![](pic.png)






![](pic.jpg)

# Dataset link:

https://www.kaggle.com/sumaiaparveenshupti/los-angeles-crime-data-20102020?select=Crime_Data_from_2020_to_Present.csv 

---

In [79]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

### Data source and Spark data abstraction (DataFrame) setup

We had to manually set the schema to avoid problems with Time column,\
Time column is in military hour, if spark infers the schema it erases the 00 at the beginning, loosing the format

In [80]:
#Defining Schema
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, DoubleType, StringType

dfschema = StructType([
    StructField("DR_NO", IntegerType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", StringType()),
    StructField("AREA", IntegerType()),      
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", IntegerType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType()),
    StructField("Premis Cd", IntegerType()),
    StructField("Premis Desc", StringType()),
    StructField("Weapon Used Cd", IntegerType()),
    StructField("Weapon Desc", StringType()),
    StructField("Status", StringType()),
    StructField("Status Desc", StringType()),
    StructField("Crm Cd 1", IntegerType()),
    StructField("Crm Cd 2", IntegerType()),
    StructField("Crm Cd 3", IntegerType()),
    StructField("Crm Cd 4", IntegerType()),
    StructField("LOCATION", StringType()),
    StructField("Cross Street", StringType()),
    StructField("LAT", DoubleType()),
    StructField("LON", DoubleType()),   
])

In [81]:
LA_crimesdf = spark.read \
                 .schema(dfschema) \
                 .option("header", "true") \
                 .csv("crime_LA.csv")
                # .option("inferSchema", "true") \

### Display schema and size of the DataFrame

In [82]:
from IPython.display import display, Markdown

LA_crimesdf.printSchema()
display(Markdown("This DataFrame has **%d rows**." % LA_crimesdf.count()))

root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: string (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 |-- LO

This DataFrame has **2117589 rows**.

In [83]:
from pyspark.sql.functions import * 

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

df_schema = LA_crimesdf.withColumn("DATE OCC", to_date(col("DATE OCC"),"MM/dd/yyyy hh:mm:ss a")) \
            .withColumn("Date Rptd", to_date(col("Date Rptd"),"MM/dd/yyyy hh:mm:ss a")) \
            .withColumn("TIME OCC", to_timestamp(col("TIME OCC"),"HHmm"))
            #.printSchema()

In [84]:
#Checking transformations
df_schema.select("Date Rptd").show(2)

+----------+
| Date Rptd|
+----------+
|2010-02-20|
|2010-09-13|
+----------+
only showing top 2 rows



In [85]:
df_schema.printSchema()

root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: date (nullable = true)
 |-- DATE OCC: date (nullable = true)
 |-- TIME OCC: timestamp (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 |-- LOC

### Random Samples

In [88]:
df_schema.cache() # optimization to make the processing faster
df_sample = df_schema.sample(False, 0.999)
display(Markdown("This DataFrame has **%d rows**." % df_sample.count()))

This DataFrame has **2115544 rows**.

In [89]:
#We do have negative ages that we will not take into consideration
df_sample=df_sample.where(col("Vict Age")>0)

display(Markdown("This DataFrame has **%d rows**." % df_sample.count()))

This DataFrame has **1745257 rows**.

In [90]:
#Checking that all the years are in the dataset
df_sample.select(year("Date Rptd")).distinct().collect()

[Row(year(Date Rptd)=2018),
 Row(year(Date Rptd)=2015),
 Row(year(Date Rptd)=2013),
 Row(year(Date Rptd)=2014),
 Row(year(Date Rptd)=2019),
 Row(year(Date Rptd)=2020),
 Row(year(Date Rptd)=2012),
 Row(year(Date Rptd)=2016),
 Row(year(Date Rptd)=2010),
 Row(year(Date Rptd)=2011),
 Row(year(Date Rptd)=2017),
 Row(year(Date Rptd)=2021)]

### Basic Profiling

**Variable segmentation:**

        DR_NO: Division of Records Number: Official file number made up of a 2 digit year, area ID, and 5 digits.

 - `Where and When Crime Ocurred`
  
        - DATE OCC: MM/DD/YYYY.
        - TIME OCC: In 24 hour military time.
        - AREA: The LAPD has 21 Community Police Stations referred to as Geographic Areas within the department. These Geographic Areas are sequentially numbered from 1-21.
        - AREA NAME: The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles.
        - Rpt Dist No: A four-digit code that represents a sub-area within a Geographic Area. All crime records reference the "RD" that it occurred in for statistical comparisons.
        - LOCATION: Street address of crime incident rounded to the nearest hundred block to maintain anonymity.
        - Cross Street: Cross Street of rounded Address.
        - LAT: Latitude.
        - LON: Longitude.
        
        
 - `Crime Description`
 
        - Crm Cd: Indicates the crime committed. (Same as Crime Code 1)
        - Crm Cd Desc: Defines the Crime Code provided.
        - Mocodes: Modus Operandi: Activities associated with the suspect in commission of the crime.
        - Premis Cd: The type of structure, vehicle, or location where the crime took place.
        - Premis Desc: Defines the Premise Code provided.
        - Weapon Used Cd: The type of weapon used in the crime.
        - Weapon Desc: Defines the Weapon Used Code provided.
        - Status: Status of the case. (IC is the default).
        - Status Desc: Defines the Status Code provided.
        - Crm Cd 1: Indicates the crime committed. Crime Code 1 is the primary and most serious one. Crime Code 2, 3, and 4 are respectively less serious offenses. Lower crime class numbers are more serious.
        - Crm Cd 2: May contain a code for an additional crime, less serious than Crime Code 1.
        - Crm Cd 3: May contain a code for an additional crime, less serious than Crime Code 1.
        - Crm Cd 4: May contain a code for an additional crime, less serious than Crime Code 1.
        
 - `Victim Description` 
        
        - Vict Age: Two character numeric.
        - Vict Sex: F - Female M - Male X - Unknown.
        - Vict Descent: Descent Code: A - Other Asian B - Black C - Chinese D - Cambodian F - Filipino G - Guamanian H - Hispanic/Latin/Mexican I - American Indian/Alaskan Native J - Japanese K - Korean L - Laotian O - Other P - Pacific Islander S - Samoan U - Hawaiian V - Vietnamese W - White X - Unknown Z - Asian Indian.
        
        

We created additional time features to help us understand the data

In [91]:
#Additional Features
from pyspark.sql.functions import year, month, quarter

df_sample=df_sample.withColumn("Year", year("Date Rptd"))\
                    .withColumn("Month",month("Date Rptd"))\
                    .withColumn("Quarter",quarter("Date Rptd"))
                    


Started the data profilling according to our variable segmentation

### Where and When Description - Variables

In [92]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit, asc_nulls_last, desc_nulls_last, hour


print('\033[1m' + "Where and When Columns" + '\033[0m')

print ("Summary of columns:")
df_sample.select(hour("TIME OCC").alias("Hour"),"Year","Month","Quarter").summary().show()

print("Checking for nulls:")
df_sample.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Date Rptd","DATE OCC","TIME OCC","AREA","AREA NAME","Rpt Dist No","LOCATION","Cross Street","LAT","LON","Year","Month","Quarter"]]).show()

print("Checking amount of distinct values:")
df_sample.select([countDistinct(c).alias(c) for c in ["AREA NAME","Rpt Dist No","Location","Cross Street"]]).show()

print ("Most and least frequent occurrences:")
freq_AreaName = df_sample.groupBy("AREA NAME").agg(count(lit(1)).alias("Total"))
freq_CrossStreet = df_sample.groupBy("Cross Street").agg(count(lit(1)).alias("Total"))
#freq_TimeOcc = df_sample.groupBy("TIME OCC").agg(count(lit(1)).alias("Total"))
freq_TimeOcc = df_sample.groupBy(hour("TIME OCC").alias("TIME OCC")).agg(count(lit(1)).alias("Total"))

leastFreqAreaName = freq_AreaName.orderBy(col("Total").asc()).first()
mostFreqAreaName = freq_AreaName.orderBy(col("Total").desc()).first()
leastFreqTimeOcc = freq_TimeOcc.orderBy(col("Total").asc()).first()
mostFreqTimeOcc = freq_TimeOcc.orderBy(col("Total").desc()).first()
leastFreqCrossStreet = freq_CrossStreet.orderBy(col("Total").asc()).first()
mostFreqCrossStreet = freq_CrossStreet.orderBy(col("Total").desc()).first()
                                               
display(Markdown(
    
    """
| %s | %s | %s | %s | %s | %s |
|----|----|----|----|----|----|
| %s | %s | %s | %s | %s | %s |
""" % ("leastFreqAreaName", "mostFreqAreaName", "leastFreqTimeOcc", "mostFreqTimeOcc", "leastFreqCrossStreet", "mostFreqCrossStreet", \
       "%s (%d occurrences)" % (leastFreqAreaName["AREA NAME"], leastFreqAreaName["Total"]), \
       "%s (%d occurrences)" % (mostFreqAreaName["AREA NAME"], mostFreqAreaName["Total"]), \
       "%s (%d occurrences)" % (leastFreqTimeOcc["TIME OCC"], leastFreqTimeOcc["Total"]), \
       "%s (%d occurrences)" % (mostFreqTimeOcc["TIME OCC"], mostFreqTimeOcc["Total"]), \
       "%s (%d occurrences)" % (leastFreqCrossStreet["Cross Street"], leastFreqCrossStreet["Total"]), \
       "%s (%d occurrences)" % (mostFreqCrossStreet["Cross Street"], mostFreqCrossStreet["Total"]))))

Where and When Columns
Summary of columns:


+-------+------------------+------------------+-----------------+------------------+
|summary|              Hour|              Year|            Month|           Quarter|
+-------+------------------+------------------+-----------------+------------------+
|  count|           1745257|           1745257|          1745257|           1745257|
|   mean|13.292080192200919|2014.6523589362484|6.525010356640885| 2.510120286009453|
| stddev| 6.453019170716877| 2.887220249709128|3.411459805471669|1.1076409331321622|
|    min|                 0|              2010|                1|                 1|
|    25%|                 9|              2012|                4|                 2|
|    50%|                14|              2015|                7|                 3|
|    75%|                19|              2017|                9|                 3|
|    max|                23|              2021|               12|                 4|
+-------+------------------+------------------+-----------------+

+---------+--------+--------+----+---------+-----------+--------+------------+---+---+----+-----+-------+
|Date Rptd|DATE OCC|TIME OCC|AREA|AREA NAME|Rpt Dist No|LOCATION|Cross Street|LAT|LON|Year|Month|Quarter|
+---------+--------+--------+----+---------+-----------+--------+------------+---+---+----+-----+-------+
|        0|       0|       0|   0|        0|          0|       0|     1455939|  0|  0|   0|    0|      0|
+---------+--------+--------+----+---------+-----------+--------+------------+---+---+----+-----+-------+

Checking amount of distinct values:


+---------+-----------+--------+------------+
|AREA NAME|Rpt Dist No|Location|Cross Street|
+---------+-----------+--------+------------+
|       21|       1285|   72276|       11374|
+---------+-----------+--------+------------+

Most and least frequent occurrences:



| leastFreqAreaName | mostFreqAreaName | leastFreqTimeOcc | mostFreqTimeOcc | leastFreqCrossStreet | mostFreqCrossStreet |
|----|----|----|----|----|----|
| Hollenbeck (57312 occurrences) | 77th Street (123682 occurrences) | 5 (21336 occurrences) | 12 (147568 occurrences) | DUBLIN                       AV (1 occurrences) | None (1455939 occurrences) |


### Crime Description - Variables

In [93]:
print('\033[1m' + "Crime Description Columns" + '\033[0m')

print ("Summary of columns:")
df_sample.select("Crm Cd","Crm Cd 1","Crm Cd 2","Crm Cd 3","Crm Cd 4").summary().show()

print("Checking for nulls:")
df_sample.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Crm Cd","Crm Cd Desc","Mocodes","Premis Cd","Premis Desc"]]).show()
df_sample.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Weapon Used Cd","Weapon Desc","Status","Status Desc","Crm Cd 1","Crm Cd 2","Crm Cd 3","Crm Cd 4"]]).show()

print("Checking amount of distinct values:")
df_sample.select([countDistinct(c).alias(c) for c in ["Weapon Used Cd","Weapon Desc","Status","Status Desc","Mocodes"]]).show()

print ("Most and least frequent occurrences:")
freq_CrmCdDesc = df_sample.groupBy("Crm Cd Desc").agg(count(lit(1)).alias("Total"))
freq_StatusDesc = df_sample.groupBy("Status Desc").agg(count(lit(1)).alias("Total"))
freq_WeaponDesc = df_sample.groupBy("Weapon Desc").agg(count(lit(1)).alias("Total"))


leastFreqCrmCdDesc = freq_CrmCdDesc.orderBy(col("Total").asc()).first()
mostFreqCrmCdDesc = freq_CrmCdDesc.orderBy(col("Total").desc()).first()
leastFreqStatusDesc = freq_StatusDesc.orderBy(col("Total").asc()).first()
mostFreqStatusDesc = freq_StatusDesc.orderBy(col("Total").desc()).first()
leastFreqWeaponDesc = freq_WeaponDesc.orderBy(col("Total").asc()).first()
mostFreqWeaponDesc = freq_WeaponDesc.orderBy(col("Total").desc()).first()

display(Markdown(
    
    """
| %s | %s | %s | %s || %s || %s |
|----|----|----|----||----||----|
| %s | %s | %s | %s || %s || %s |
""" % ("leastFreqCrmCdDesc", "mostFreqCrmCdDesc", "leastFreqStatusDesc", "mostFreqStatusDesc","leastFreqWeaponDesc", "mostFreqWeaponDesc", \
       "%s (%d occurrences)" % (leastFreqCrmCdDesc["Crm Cd Desc"], leastFreqCrmCdDesc["Total"]), \
       "%s (%d occurrences)" % (mostFreqCrmCdDesc["Crm Cd Desc"], mostFreqCrmCdDesc["Total"]), \
       "%s (%d occurrences)" % (leastFreqStatusDesc["Status Desc"], leastFreqStatusDesc["Total"]), \
       "%s (%d occurrences)" % (mostFreqStatusDesc["Status Desc"], mostFreqStatusDesc["Total"]), \
       "%s (%d occurrences)" % (leastFreqWeaponDesc["Weapon Desc"], leastFreqWeaponDesc["Total"]), \
       "%s (%d occurrences)" % (mostFreqWeaponDesc["Weapon Desc"], mostFreqWeaponDesc["Total"]))))


Crime Description Columns
Summary of columns:


+-------+-----------------+------------------+-----------------+-----------------+-----------------+
|summary|           Crm Cd|          Crm Cd 1|         Crm Cd 2|         Crm Cd 3|         Crm Cd 4|
+-------+-----------------+------------------+-----------------+-----------------+-----------------+
|  count|          1745257|           1745248|           114819|             3240|               99|
|   mean|506.1865559055199|505.98993781972536|943.2570306308189|972.5608024691358|975.3434343434343|
| stddev|220.4244607410514| 220.2705380127317|131.5103035547205|85.25279775547192|82.37856388700237|
|    min|              110|               110|              210|               93|              421|
|    25%|              330|               330|              998|              998|              998|
|    50%|              440|               440|              998|              998|              998|
|    75%|              648|               648|              998|              998|         

+--------------+-----------+------+-----------+--------+--------+--------+--------+
|Weapon Used Cd|Weapon Desc|Status|Status Desc|Crm Cd 1|Crm Cd 2|Crm Cd 3|Crm Cd 4|
+--------------+-----------+------+-----------+--------+--------+--------+--------+
|       1069367|    1069368|     1|          0|       9| 1630438| 1742017| 1745158|
+--------------+-----------+------+-----------+--------+--------+--------+--------+

Checking amount of distinct values:


+--------------+-----------+------+-----------+-------+
|Weapon Used Cd|Weapon Desc|Status|Status Desc|Mocodes|
+--------------+-----------+------+-----------+-------+
|            80|         79|     6|          6| 453268|
+--------------+-----------+------+-----------+-------+

Most and least frequent occurrences:



| leastFreqCrmCdDesc | mostFreqCrmCdDesc | leastFreqStatusDesc | mostFreqStatusDesc || leastFreqWeaponDesc || mostFreqWeaponDesc |
|----|----|----|----||----||----|
| FIREARMS TEMPORARY RESTRAINING ORDER (TEMP FIREARMS RO) (1 occurrences) | BATTERY - SIMPLE ASSAULT (188537 occurrences) | UNK (25 occurrences) | Invest Cont (1308566 occurrences) || M-14 SEMIAUTOMATIC ASSAULT RIFLE (1 occurrences) || None (1069368 occurrences) |


### Victim Description - Variables

In [94]:
#Victim Description

print('\033[1m' + "Victim Description Columns" + '\033[0m')

print ("Summary of columns:")
df_sample.select("Vict Age").summary().show()

print("Checking for nulls:")
df_sample.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Vict Age","Vict Sex","Vict Descent"]]).show()

print("Checking amount of distinct values:")
df_sample.select([countDistinct(c).alias(c) for c in ["Vict Age","Vict Sex","Vict Descent"]]).show()


print ("Most and least frequent occurrences:")
freq_VictAge = df_sample.groupBy("Vict Age").agg(count(lit(1)).alias("Total"))
freq_VictSex = df_sample.groupBy("Vict Sex").agg(count(lit(1)).alias("Total"))
freq_VictDescent = df_sample.groupBy("Vict Descent").agg(count(lit(1)).alias("Total"))


leastFreqVictAge = freq_VictAge.orderBy(col("Total").asc()).first()
mostFreqVictAge = freq_VictAge.orderBy(col("Total").desc()).first()
leastFreqVictSex = freq_VictSex.orderBy(col("Total").asc()).first()
mostFreqVictSex = freq_VictSex.orderBy(col("Total").desc()).first()
leastFreqVictDescent = freq_VictDescent.orderBy(col("Total").asc()).first()
mostFreqVictDescent = freq_VictDescent.orderBy(col("Total").desc()).first()
                                               
display(Markdown(
    
    """
| %s | %s | %s | %s | %s | %s |
|----|----|----|----|----|----|
| %s | %s | %s | %s | %s | %s |
""" % ("leastFreqVictAge", "mostFreqVictAge", "leastFreqVictSex", "mostFreqVictSex","leastFreqVictDescen", "mostFreqVictDescen", \
       "%s (%d occurrences)" % (leastFreqVictAge["Vict Age"], leastFreqVictAge["Total"]), \
       "%s (%d occurrences)" % (mostFreqVictAge["Vict Age"], mostFreqVictAge["Total"]), \
       "%s (%d occurrences)" % (leastFreqVictSex["Vict Sex"], leastFreqVictSex["Total"]), \
       "%s (%d occurrences)" % (mostFreqVictSex["Vict Sex"], mostFreqVictSex["Total"]),\
       "%s (%d occurrences)" % (leastFreqVictDescent["Vict Descent"], leastFreqVictDescent["Total"]), \
       "%s (%d occurrences)" % (mostFreqVictDescent["Vict Descent"], mostFreqVictDescent["Total"]))))

Victim Description Columns
Summary of columns:


+-------+------------------+
|summary|          Vict Age|
+-------+------------------+
|  count|           1745257|
|   mean| 38.49861481718738|
| stddev|16.039815794776754|
|    min|                 2|
|    25%|                26|
|    50%|                36|
|    75%|                50|
|    max|               118|
+-------+------------------+

Checking for nulls:
+--------+--------+------------+
|Vict Age|Vict Sex|Vict Descent|
+--------+--------+------------+
|       0|      66|          94|
+--------+--------+------------+

Checking amount of distinct values:


+--------+--------+------------+
|Vict Age|Vict Sex|Vict Descent|
+--------+--------+------------+
|     100|       6|          20|
+--------+--------+------------+

Most and least frequent occurrences:



| leastFreqVictAge | mostFreqVictAge | leastFreqVictSex | mostFreqVictSex | leastFreqVictDescen | mostFreqVictDescen |
|----|----|----|----|----|----|
| 114 (1 occurrences) | 25 (48105 occurrences) | - (1 occurrences) | F (876564 occurrences) | - (2 occurrences) | H (704340 occurrences) |


# Business Questions

## A. Ratio of crimes commited by time segment

In [95]:
from pyspark.sql.functions import count, round

# Time Segmentation:
#
#   "Early Morning"     [5:00 AM,9:00 AM) 
#   "Late Morning"     [9:00 AM,12:00 AM)
#   "Afternoon"         [12:00 AM,15:00 PM)
#   "Evening"    (15:00 PM,20:00 PM]
#   "Night"     (20:00 PM,00:00 AM)
#   "Dawn"     [00:00 AM,5:00 AM)



# 1. Let's enrich the DF with our categorization
totalCrimes = df_sample.count()
TimeCategorizationDF = df_sample.withColumn("TimeSegment", when(((hour(col("TIME OCC"))>=5) & (hour(col("TIME OCC"))<9)).alias("TIME OCC"),"1.Early Morning")\
                               .when(((hour(col("TIME OCC"))>=9) & (hour(col("TIME OCC"))<12)).alias("TIME OCC"),"2.Late Morning")\
                               .when(((hour(col("TIME OCC"))>=12) & (hour(col("TIME OCC"))<15)).alias("TIME OCC"),"3.Afternoon")\
                               .when(((hour(col("TIME OCC"))>=15) & (hour(col("TIME OCC"))<20)).alias("TIME OCC"),"4.Evening")\
                               .when(((hour(col("TIME OCC"))>=20) & (hour(col("TIME OCC"))<=23)).alias("TIME OCC"),"5.Night")\
                               .otherwise("6.Dawn"))
TimeCategorizationDF.cache() # optimization to make the processing faster

# 2. Ready to answer to this business question
TimeCategorizationDF.select("TimeSegment", "TIME OCC")\
                     .groupBy("TimeSegment")\
                     .agg(count("TimeSegment").alias("NumCrimes"), \
                          (count("TimeSegment")/totalCrimes*100).alias("Ratio"))\
                     .orderBy("TimeSegment")\
                     .select("TimeSegment","NumCrimes",round("Ratio",2).alias("RoundedRatio")).show()

+---------------+---------+------------+
|    TimeSegment|NumCrimes|RoundedRatio|
+---------------+---------+------------+
|1.Early Morning|   172982|        9.91|
| 2.Late Morning|   219416|       12.57|
|    3.Afternoon|   309670|       17.74|
|      4.Evening|   467938|       26.81|
|        5.Night|   351276|       20.13|
|         6.Dawn|   223975|       12.83|
+---------------+---------+------------+



In [96]:
from pyspark.sql.functions import count, round

# Crime Severity Segmentation:
#
#   "Severe"     [0, 200] 
#   "Less Severe"     (200,400]
#   "Moderate"        (400,600]
#   "Mild"            (600,800]
#   "Not Severe"     (800,1000)




# 1. Let's enrich the DF with our categorization
totalCrimes = df_sample.count()
SeverityCategorizationDF = TimeCategorizationDF.withColumn("CrimeSeverity", when(col("Crm Cd")<=200,"1.Severe")\
                               .when((col("Crm Cd")>200) & (col("Crm Cd")<=400),"2.Less Severe")\
                               .when((col("Crm Cd")>400) & (col("Crm Cd")<600),"3.Moderate")\
                               .when((col("Crm Cd")>600) & (col("Crm Cd")<800),"4.Mild")\
                               .otherwise("5.Not Severe"))
SeverityCategorizationDF.cache() # optimization to make the processing faster

# 2. Ready to answer to this business question
SeverityCategorizationDF.select("CrimeSeverity", "Crm Cd")\
                     .groupBy("CrimeSeverity")\
                     .agg(count("CrimeSeverity").alias("NumCrimes"), \
                          (count("CrimeSeverity")/totalCrimes*100).alias("Ratio"))\
                     .orderBy("CrimeSeverity")\
                     .select("CrimeSeverity","NumCrimes",round("Ratio",2).alias("RoundedRatio")).show()

+-------------+---------+------------+
|CrimeSeverity|NumCrimes|RoundedRatio|
+-------------+---------+------------+
|     1.Severe|    14212|        0.81|
|2.Less Severe|   737762|       42.27|
|   3.Moderate|   236326|       13.54|
|       4.Mild|   551696|       31.61|
| 5.Not Severe|   205261|       11.76|
+-------------+---------+------------+



In [97]:
display(Markdown("**Severity of Crime by Time of the day** (Order by 1.Severe):"))

SeverityCategorizationDF.groupBy("TimeSegment")\
                        .pivot("CrimeSeverity")\
                        .agg(count("CrimeSeverity")/totalCrimes*100)\
                        .orderBy(col("`1.Severe`").desc())\
                        .select("TimeSegment",round("`1.Severe`",2).alias("1.Severe"),round("`2.Less Severe`",2).alias("2.Less Severe"),round("`3.Moderate`",2).alias("3.Moderate"),round("`4.Mild`",2).alias("4.Mild"),round("`5.Not Severe`",2).alias("5. Not Severe")).show()

**Severity of Crime by Time of the day** (Order by 1.Severe):

+---------------+--------+-------------+----------+------+-------------+
|    TimeSegment|1.Severe|2.Less Severe|3.Moderate|4.Mild|5. Not Severe|
+---------------+--------+-------------+----------+------+-------------+
|         6.Dawn|    0.26|         5.65|      1.21|  4.41|          1.3|
|        5.Night|     0.2|         8.27|      2.43|  7.21|         2.01|
|      4.Evening|    0.14|        10.68|      4.28|  8.52|         3.18|
|    3.Afternoon|    0.09|         7.86|      2.67|  4.86|         2.27|
|1.Early Morning|    0.07|         4.58|      1.16|   2.8|          1.3|
| 2.Late Morning|    0.05|         5.23|      1.79|  3.81|          1.7|
+---------------+--------+-------------+----------+------+-------------+



In [98]:
display(Markdown("**Severity of Crime by Time of the day** (Order by 3.Moderate):"))

#,col("`2.Less Severe`").desc(),col("`3.Moderate`").desc(),col("`4.Mild`").desc(),col("`5.Not Severe`").desc()

SeverityCategorizationDF.groupBy("TimeSegment")\
                        .pivot("CrimeSeverity")\
                        .agg(count("CrimeSeverity")/totalCrimes*100)\
                        .orderBy(col("`3.Moderate`").desc())\
                        .select("TimeSegment",round("`1.Severe`",2).alias("1.Severe"),round("`2.Less Severe`",2).alias("2.Less Severe"),round("`3.Moderate`",2).alias("3.Moderate"),round("`4.Mild`",2).alias("4.Mild"),round("`5.Not Severe`",2).alias("5. Not Severe")).show()

**Severity of Crime by Time of the day** (Order by 3.Moderate):

+---------------+--------+-------------+----------+------+-------------+
|    TimeSegment|1.Severe|2.Less Severe|3.Moderate|4.Mild|5. Not Severe|
+---------------+--------+-------------+----------+------+-------------+
|      4.Evening|    0.14|        10.68|      4.28|  8.52|         3.18|
|    3.Afternoon|    0.09|         7.86|      2.67|  4.86|         2.27|
|        5.Night|     0.2|         8.27|      2.43|  7.21|         2.01|
| 2.Late Morning|    0.05|         5.23|      1.79|  3.81|          1.7|
|         6.Dawn|    0.26|         5.65|      1.21|  4.41|          1.3|
|1.Early Morning|    0.07|         4.58|      1.16|   2.8|          1.3|
+---------------+--------+-------------+----------+------+-------------+



In [99]:
display(Markdown("**Severity of Crime by Time of the day** (Order by 5.Not Severe):"))

SeverityCategorizationDF.groupBy("TimeSegment")\
                        .pivot("CrimeSeverity")\
                        .agg(count("CrimeSeverity")/totalCrimes*100)\
                        .orderBy(col("`5.Not Severe`").desc())\
                        .select("TimeSegment",round("`1.Severe`",2).alias("1.Severe"),round("`2.Less Severe`",2).alias("2.Less Severe"),round("`3.Moderate`",2).alias("3.Moderate"),round("`4.Mild`",2).alias("4.Mild"),round("`5.Not Severe`",2).alias("5. Not Severe")).show()

**Severity of Crime by Time of the day** (Order by 5.Not Severe):

+---------------+--------+-------------+----------+------+-------------+
|    TimeSegment|1.Severe|2.Less Severe|3.Moderate|4.Mild|5. Not Severe|
+---------------+--------+-------------+----------+------+-------------+
|      4.Evening|    0.14|        10.68|      4.28|  8.52|         3.18|
|    3.Afternoon|    0.09|         7.86|      2.67|  4.86|         2.27|
|        5.Night|     0.2|         8.27|      2.43|  7.21|         2.01|
| 2.Late Morning|    0.05|         5.23|      1.79|  3.81|          1.7|
|         6.Dawn|    0.26|         5.65|      1.21|  4.41|          1.3|
|1.Early Morning|    0.07|         4.58|      1.16|   2.8|          1.3|
+---------------+--------+-------------+----------+------+-------------+



In [100]:
display(Markdown("**Severity of Crime by Year**"))

SeverityCategorizationDF.groupBy("Year")\
                        .pivot("CrimeSeverity")\
                        .agg(count("CrimeSeverity"))\
                        .orderBy(col("Year").desc())\
                        .show()

**Severity of Crime by Year**

+----+--------+-------------+----------+------+------------+
|Year|1.Severe|2.Less Severe|3.Moderate|4.Mild|5.Not Severe|
+----+--------+-------------+----------+------+------------+
|2021|      84|          563|        23|    92|         393|
|2020|     251|         2743|       472|   839|        1351|
|2019|    1465|        72225|     25451| 58363|       20893|
|2018|    1628|        77831|     26428| 59995|       21827|
|2017|    1698|        79117|     25716| 59495|       21119|
|2016|    1638|        77376|     26710| 57828|       20427|
|2015|    1591|        75016|     27015| 55134|       19834|
|2014|    1309|        66160|     23876| 52535|       19467|
|2013|    1072|        67832|     22962| 50144|       18877|
|2012|    1227|        71033|     22467| 52361|       20390|
|2011|    1091|        71617|     20186| 51957|       20357|
|2010|    1158|        76249|     15020| 52953|       20326|
+----+--------+-------------+----------+------+------------+



### Conclusion:
\
With this results we can conclude that there is a tendency of having severe crimes ocurre during dawn and night, moderate and not severre crimes ocurre during the evening or the afternoon leaving the morning as the safest time of the day.\
\
We can also conclude that crimes have mantained constant during the last decade having an exception in 2020 mostly related to covid impact. 

### B. Is there Age, Gender or Race Discrimination by Crime Severity?

**B.1 Gender**

In [101]:
TotalCrimeMale = df_sample.where(col("Vict Sex")=="M").count()

TotalCrimeFemale = df_sample.where(col("Vict Sex")=="F").count()

print("Total Crimes towards Male",TotalCrimeMale)
print("Total Crimes towards Female", TotalCrimeFemale)
print("Ratio Male/Women:", TotalCrimeMale/TotalCrimeFemale)

Total Crimes towards Male 862581
Total Crimes towards Female 876564
Ratio Male/Women: 0.9840479417361425


In [102]:
from pyspark.sql.functions import max, min, avg, stddev

TotalCrimeMale = df_sample.where(col("Vict Sex")=="M").count()

display(Markdown("**Severity of Crime by Gender** (MALE):"))
SeverityCategorizationDF.where(col("Vict Sex")=="M")\
              .groupBy("CrimeSeverity")\
              .agg(count("CrimeSeverity").alias("TotalCrimesGender"), \
                  (count("CrimeSeverity")/TotalCrimeMale*100).alias("Ratio"))\
               .select("CrimeSeverity","TotalCrimesGender",round("Ratio",2).alias("RoundedRatio")).show()

display(Markdown("With total crimes of **%d**." % TotalCrimeMale))

**Severity of Crime by Gender** (MALE):

+-------------+-----------------+------------+
|CrimeSeverity|TotalCrimesGender|RoundedRatio|
+-------------+-----------------+------------+
|     1.Severe|             2457|        0.28|
|2.Less Severe|           416118|       48.24|
|   3.Moderate|           129574|       15.02|
| 5.Not Severe|            71889|        8.33|
|       4.Mild|           242543|       28.12|
+-------------+-----------------+------------+



With total crimes of **862581**.

In [103]:
from pyspark.sql.functions import max, min, avg, stddev

display(Markdown("**Severity of Crime by Gender** (FEMALE):"))
SeverityCategorizationDF.where(col("Vict Sex")=="F")\
              .groupBy("CrimeSeverity")\
              .agg(count("CrimeSeverity").alias("TotalCrimesGender"), \
                  (count("CrimeSeverity")/TotalCrimeMale*100).alias("Ratio"))\
               .select("CrimeSeverity","TotalCrimesGender",round("Ratio",2).alias("RoundedRatio")).show()

display(Markdown("With total crimes of **%d**." % TotalCrimeFemale))

**Severity of Crime by Gender** (FEMALE):

+-------------+-----------------+------------+
|CrimeSeverity|TotalCrimesGender|RoundedRatio|
+-------------+-----------------+------------+
|     1.Severe|            11753|        1.36|
|2.Less Severe|           319764|       37.07|
|   3.Moderate|           104873|       12.16|
| 5.Not Severe|           132692|       15.38|
|       4.Mild|           307482|       35.65|
+-------------+-----------------+------------+



With total crimes of **876564**.

In [133]:
import pyspark.sql.functions as F

display(Markdown("**Most Common Crimes by Sex**"))

SeverityCategorizationDF.where((col("Vict Sex")=="F") | (col("Vict Sex")=="M"))\
                        .groupBy("Crm Cd Desc")\
                        .pivot("Vict Sex")\
                        .agg(count("Vict Sex"))\
                        .withColumn("F/M ratio",F.col("F")/F.col("M"))\
                        .orderBy(col("F/M ratio").desc())\
                        .select("Crm Cd Desc","F","M",round("F/M ratio",2).alias("Ratio")).show(truncate = False)

**Most Common Crimes by Sex**

+--------------------------------------------------------+-----+----+------+
|Crm Cd Desc                                             |F    |M   |Ratio |
+--------------------------------------------------------+-----+----+------+
|RAPE, FORCIBLE                                          |10293|86  |119.69|
|RAPE, ATTEMPTED                                         |1092 |13  |84.0  |
|HUMAN TRAFFICKING - COMMERCIAL SEX ACTS                 |465  |7   |66.43 |
|PIMPING                                                 |449  |8   |56.13 |
|PURSE SNATCHING - ATTEMPT                               |45   |2   |22.5  |
|PURSE SNATCHING                                         |1115 |63  |17.7  |
|HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE               |91   |6   |15.17 |
|BATTERY WITH SEXUAL CONTACT                             |10322|925 |11.16 |
|PEEPING TOM                                             |1048 |103 |10.17 |
|SEXUAL PENETRATION W/FOREIGN OBJECT                     |2463 |244 |10.09 |

### B.1 Conclusion:
\
Although there does not seem to be discrimination by gender in general terms, when looking at the specific severnes of the crimes female are almost 100 times more likely to be victims of severe crime than men.

**B.2 Age**

In [105]:
from pyspark.sql.functions import max, min, avg, stddev

display(Markdown("**Severity of Crime by Age**"))
SeverityCategorizationDF.groupBy("CrimeSeverity")\
              .agg(avg("Vict Age").alias("AverageAge"),\
                   min("Vict Age").alias("LowestAge"),\
                   max("Vict Age").alias("HighestAge"),\
                   stddev("Vict Age").alias("StdDevAge"))\
              .orderBy("CrimeSeverity").show()

**Severity of Crime by Age**

+-------------+------------------+---------+----------+------------------+
|CrimeSeverity|        AverageAge|LowestAge|HighestAge|         StdDevAge|
+-------------+------------------+---------+----------+------------------+
|     1.Severe|28.372924289332957|        2|        99|13.565644280848735|
|2.Less Severe|40.034988248242655|        2|       118| 16.07433332879085|
|   3.Moderate|38.935927489992636|        2|        99|15.670289478608785|
|       4.Mild|37.749494286708625|        2|        99|15.612719071862905|
| 5.Not Severe| 35.18754658702822|        2|        99| 16.73079112178732|
+-------------+------------------+---------+----------+------------------+



In [106]:
from pyspark.sql.functions import count, round

# Age Segmentation:
#
#   [0-18)
#   [18-25)
#   [25-35)
#   [35-45)
#   [45-55)
#   [55-65)
#   [65 and older




# 1. Let's enrich the DF with our categorization
totalCrimes = df_sample.count()
SeverityCategorizationDF = SeverityCategorizationDF.withColumn("Age_Seg", when(col("Vict Age")<18,"0-17")\
                               .when((col("Vict Age")>=18) & (col("Vict Age")<25),"18-24")\
                               .when((col("Vict Age")>=25) & (col("Vict Age")<35),"25-34")\
                               .when((col("Vict Age")>=35) & (col("Vict Age")<45),"35-44")\
                               .when((col("Vict Age")>=45) & (col("Vict Age")<55),"45-54")\
                               .when((col("Vict Age")>=55) & (col("Vict Age")<65),"55-64")\
                               .otherwise("65+"))
SeverityCategorizationDF.cache() # optimization to make the processing faster

# 2. Ready to answer to this business question
display(Markdown("**Severity of Crime by Age Segmentation**"))

SeverityCategorizationDF.groupBy("Age_Seg")\
                        .pivot("CrimeSeverity")\
                        .agg(count("CrimeSeverity"))\
                        .orderBy(col("Age_Seg"))\
                        .show()

**Severity of Crime by Age Segmentation**

+-------+--------+-------------+----------+------+------------+
|Age_Seg|1.Severe|2.Less Severe|3.Moderate|4.Mild|5.Not Severe|
+-------+--------+-------------+----------+------+------------+
|   0-17|    2843|        27029|      8180| 31902|       33816|
|  18-24|    4023|       101649|     38421| 88299|       26006|
|  25-34|    3546|       187816|     62477|145806|       46547|
|  35-44|    1841|       151090|     46787|108587|       39688|
|  45-54|    1208|       126814|     38733| 90976|       31603|
|  55-64|     536|        84670|     25447| 55171|       17548|
|    65+|     215|        58694|     16281| 30955|       10053|
+-------+--------+-------------+----------+------+------------+



In [134]:
display(Markdown("**Severity of Crime by Age** (Order by 1.Severe):"))
SeverityCategorizationDF.groupBy("Age_Seg")\
                        .pivot("CrimeSeverity")\
                        .agg(count("CrimeSeverity")/totalCrimes*100)\
                        .orderBy(col("`1.Severe`").desc())\
                        .select("Age_Seg",round("`1.Severe`",2).alias("1.Severe"),round("`2.Less Severe`",2).alias("2.Less Severe"),round("`3.Moderate`",2).alias("3.Moderate"),round("`4.Mild`",2).alias("4.Mild"),round("`5.Not Severe`",2).alias("5. Not Severe")).show()

**Severity of Crime by Age** (Order by 1.Severe):

+-------+--------+-------------+----------+------+-------------+
|Age_Seg|1.Severe|2.Less Severe|3.Moderate|4.Mild|5. Not Severe|
+-------+--------+-------------+----------+------+-------------+
|  18-24|    0.23|         5.82|       2.2|  5.06|         1.49|
|  25-34|     0.2|        10.76|      3.58|  8.35|         2.67|
|   0-17|    0.16|         1.55|      0.47|  1.83|         1.94|
|  35-44|    0.11|         8.66|      2.68|  6.22|         2.27|
|  45-54|    0.07|         7.27|      2.22|  5.21|         1.81|
|  55-64|    0.03|         4.85|      1.46|  3.16|         1.01|
|    65+|    0.01|         3.36|      0.93|  1.77|         0.58|
+-------+--------+-------------+----------+------+-------------+



In [108]:
display(Markdown("**Severity of Crime by Age** (Order by 3.Moderate):"))
SeverityCategorizationDF.groupBy("Age_Seg")\
                        .pivot("CrimeSeverity")\
                        .agg(count("CrimeSeverity")/totalCrimes*100)\
                        .orderBy(col("`3.Moderate`").desc())\
                        .select("Age_Seg",round("`1.Severe`",2).alias("1.Severe"),round("`2.Less Severe`",2).alias("2.Less Severe"),round("`3.Moderate`",2).alias("3.Moderate"),round("`4.Mild`",2).alias("4.Mild"),round("`5.Not Severe`",2).alias("5. Not Severe")).show()

**Severity of Crime by Age** (Order by 3.Moderate):

+-------+--------+-------------+----------+------+-------------+
|Age_Seg|1.Severe|2.Less Severe|3.Moderate|4.Mild|5. Not Severe|
+-------+--------+-------------+----------+------+-------------+
|  25-34|     0.2|        10.76|      3.58|  8.35|         2.67|
|  35-44|    0.11|         8.66|      2.68|  6.22|         2.27|
|  45-54|    0.07|         7.27|      2.22|  5.21|         1.81|
|  18-24|    0.23|         5.82|       2.2|  5.06|         1.49|
|  55-64|    0.03|         4.85|      1.46|  3.16|         1.01|
|    65+|    0.01|         3.36|      0.93|  1.77|         0.58|
|   0-17|    0.16|         1.55|      0.47|  1.83|         1.94|
+-------+--------+-------------+----------+------+-------------+



In [135]:
display(Markdown("**Severity of Crime by Age** (Order by 5.Not Severe):"))
SeverityCategorizationDF.groupBy("Age_Seg")\
                        .pivot("CrimeSeverity")\
                        .agg(count("CrimeSeverity")/totalCrimes*100)\
                        .orderBy(col("`5.Not Severe`").desc())\
                        .select("Age_Seg",round("`1.Severe`",2).alias("1.Severe"),round("`2.Less Severe`",2).alias("2.Less Severe"),round("`3.Moderate`",2).alias("3.Moderate"),round("`4.Mild`",2).alias("4.Mild"),round("`5.Not Severe`",2).alias("5. Not Severe")).show()

**Severity of Crime by Age** (Order by 5.Not Severe):

+-------+--------+-------------+----------+------+-------------+
|Age_Seg|1.Severe|2.Less Severe|3.Moderate|4.Mild|5. Not Severe|
+-------+--------+-------------+----------+------+-------------+
|  25-34|     0.2|        10.76|      3.58|  8.35|         2.67|
|  35-44|    0.11|         8.66|      2.68|  6.22|         2.27|
|   0-17|    0.16|         1.55|      0.47|  1.83|         1.94|
|  45-54|    0.07|         7.27|      2.22|  5.21|         1.81|
|  18-24|    0.23|         5.82|       2.2|  5.06|         1.49|
|  55-64|    0.03|         4.85|      1.46|  3.16|         1.01|
|    65+|    0.01|         3.36|      0.93|  1.77|         0.58|
+-------+--------+-------------+----------+------+-------------+



### B.2 Conclusion:
\
Babies and Teenagers seem to be propense to be victims of severe crimes in comparison to other ages, while adults between 25 and 34 are the most common victims of moderate and not severe crimes.

**B.3 Race**

In [110]:
display(Markdown("**Severity of Crime by Race**"))

#Descent Code: 
#A - Other Asian 
#B - Black 
#C - Chinese 
#D - Cambodian 
#F - Filipino 
#G - Guamanian 
#H - Hispanic/Latin/Mexican 
#I - American Indian/Alaskan Native 
#J - Japanese 
#K - Korean 
#L - Laotian 
#O - Other 
#P - Pacific Islander 
#S - Samoan 
#U - Hawaiian 
#V - Vietnamese 
#W - White 
#X - Unknown 
#Z - Asian Indian.


# 1. Let's enrich the DF with our categorization
SeverityCategorizationDF = SeverityCategorizationDF.withColumn("Race Desc", when(col("Vict Descent")=="A","Other Asian")\
                               .when(col("Vict Descent")=="B","Black")\
                               .when(col("Vict Descent")=="C","Chinese")\
                               .when(col("Vict Descent")=="D","Cambodian")\
                               .when(col("Vict Descent")=="F","Filipino")\
                               .when(col("Vict Descent")=="G","Guamanian")\
                               .when(col("Vict Descent")=="H","Hispanic/Latin/Mexican")\
                               .when(col("Vict Descent")=="I","American Indian/Alaskan Native")\
                               .when(col("Vict Descent")=="J","Japanese")\
                               .when(col("Vict Descent")=="K","Korean")\
                               .when(col("Vict Descent")=="L","Laotian")\
                               .when(col("Vict Descent")=="O","Other")\
                               .when(col("Vict Descent")=="P","Pacific Islander")\
                               .when(col("Vict Descent")=="S","Samoan")\
                               .when(col("Vict Descent")=="U","Hawaiian")\
                               .when(col("Vict Descent")=="V","Vietnamese")\
                               .when(col("Vict Descent")=="W","White")\
                               .when(col("Vict Descent")=="Z","Asian Indian")\
                               .otherwise("Unknown"))


SeverityCategorizationDF.groupBy("Race Desc")\
                        .pivot("CrimeSeverity")\
                        .agg(count("CrimeSeverity"))\
                        .orderBy(col("`1.Severe`").desc(),col("`2.Less Severe`").desc(),col("`3.Moderate`").desc(),col("`4.Mild`").desc(),col("`5.Not Severe`").desc())\
                        .show(truncate = False)

**Severity of Crime by Race**

+------------------------------+--------+-------------+----------+------+------------+
|Race Desc                     |1.Severe|2.Less Severe|3.Moderate|4.Mild|5.Not Severe|
+------------------------------+--------+-------------+----------+------+------------+
|Hispanic/Latin/Mexican        |6137    |273862       |84340     |245941|94060       |
|Black                         |3736    |126661       |35671     |122643|40774       |
|White                         |3234    |220433       |74746     |119904|48930       |
|Other                         |698     |79429        |25343     |43978 |15325       |
|Other Asian                   |297     |25768        |8447      |12044 |3519        |
|Korean                        |42      |4389         |1022      |2487  |696         |
|Filipino                      |27      |1150         |393       |696   |273         |
|Unknown                       |21      |4582         |5502      |3274  |1440        |
|American Indian/Alaskan Native|8       |30

In [111]:
display(Markdown("**Most Common Crimes against Hispanic/Latin/Mexican**"))

TotalCrimeHisp = SeverityCategorizationDF.where(col("Race Desc")=="Hispanic/Latin/Mexican").count()


SeverityCategorizationDF.where(col("Race Desc")=="Hispanic/Latin/Mexican")\
                        .groupBy("Crm Cd Desc")\
                        .agg(count("Crm Cd Desc").alias("Total Crimes"))\
                        .orderBy(col("Total Crimes").desc())\
                        .select("Crm Cd Desc","Total Crimes",round(col("Total Crimes")/TotalCrimeHisp,2).alias("Ratio")).show(truncate = False)

**Most Common Crimes against Hispanic/Latin/Mexican**

+--------------------------------------------------------+------------+-----+
|Crm Cd Desc                                             |Total Crimes|Ratio|
+--------------------------------------------------------+------------+-----+
|BATTERY - SIMPLE ASSAULT                                |85095       |0.12 |
|INTIMATE PARTNER - SIMPLE ASSAULT                       |58833       |0.08 |
|BURGLARY FROM VEHICLE                                   |56708       |0.08 |
|ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT          |44261       |0.06 |
|THEFT PLAIN - PETTY ($950 & UNDER)                      |42152       |0.06 |
|ROBBERY                                                 |41705       |0.06 |
|THEFT OF IDENTITY                                       |40544       |0.06 |
|VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS) |37487       |0.05 |
|BURGLARY                                                |35077       |0.05 |
|VANDALISM - MISDEAMEANOR ($399 OR UNDER)                |32943 

In [112]:
display(Markdown("**Most Common Area where Crimes ocurr against Hispanic/Latin/Mexican**"))

TotalCrimeHisp = SeverityCategorizationDF.where(col("Race Desc")=="Hispanic/Latin/Mexican").count()


SeverityCategorizationDF.where(col("Race Desc")=="Hispanic/Latin/Mexican")\
                        .groupBy("AREA NAME")\
                        .agg(count("AREA NAME").alias("Total Crimes"))\
                        .orderBy(col("Total Crimes").desc())\
                        .select("AREA NAME","Total Crimes",round(col("Total Crimes")/TotalCrimeHisp,2).alias("Ratio")).show(truncate = False)

**Most Common Area where Crimes ocurr against Hispanic/Latin/Mexican**

+-----------+------------+-----+
|AREA NAME  |Total Crimes|Ratio|
+-----------+------------+-----+
|Mission    |57755       |0.08 |
|Newton     |50934       |0.07 |
|77th Street|49021       |0.07 |
|Hollenbeck |47171       |0.07 |
|Rampart    |44240       |0.06 |
|Southeast  |42864       |0.06 |
|Southwest  |40997       |0.06 |
|Foothill   |38472       |0.05 |
|Northeast  |36313       |0.05 |
|Harbor     |36169       |0.05 |
|Olympic    |35255       |0.05 |
|N Hollywood|30272       |0.04 |
|Central    |29317       |0.04 |
|Van Nuys   |29268       |0.04 |
|Topanga    |24567       |0.03 |
|Devonshire |24251       |0.03 |
|West Valley|23393       |0.03 |
|Hollywood  |20087       |0.03 |
|Wilshire   |18002       |0.03 |
|Pacific    |16300       |0.02 |
+-----------+------------+-----+
only showing top 20 rows



### B.3 Conclusion:
\
One of the most interesting insights is the fact that hispanics are the favorite race for crimes in LA, doubling the amount of crimes of the next race (Black) in all of the severness segment. They are victims of simple crimes like simple assaults to burglary from vehicle assault. The most common places where latins are victims of crime are Mission, Newton, 77th Street and Hollenbeck.

\
Given that in our group we are mostly latins, we decided to go deeper and understand the motives

# Motive

To answer this question we had to search for the internaitonal codes for motives because our dataset did not have the description of the codes. Then we joined the data, did more feature engineering and decided to train a model to see if this features where enough to generate a reliable predictor to know if a crime was being committed to a hispanic or not.

In [113]:
Mocodes_df = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("MO_Codes.csv")

In [114]:
Mocodes_df.printSchema()
display(Markdown("This DataFrame has **%d rows**." % Mocodes_df.count()))

root
 |-- MO_CODE: integer (nullable = true)
 |-- MO_DESCRIPTION: string (nullable = true)



This DataFrame has **824 rows**.

In [115]:
SeverityCategorizationDF.select("DR_NO","Mocodes").distinct().show(truncate = False)

+---------+-------------------+
|DR_NO    |Mocodes            |
+---------+-------------------+
|100104832|0344               |
|100104836|0100               |
|100105037|0416               |
|100105253|0417 2000          |
|100107036|0337 0421 0543 1240|
|100107054|0344               |
|100107609|0400 1217 1251     |
|100107942|0601               |
|100108114|0400               |
|100109240|0416 0520          |
|100109261|0329 1307          |
|100110234|0344 1218 2004     |
|100113380|0400 0431 1027 1202|
|100113782|0416 2000          |
|100114100|0344 1307 1609     |
|100115187|0421               |
|100115926|0344 1251          |
|100116221|0416               |
|100117462|0416               |
|100118090|1501               |
+---------+-------------------+
only showing top 20 rows



In [116]:
Mocodes_split_df= SeverityCategorizationDF.select(
    "DR_NO","Race Desc",
    F.split("Mocodes"," ").alias("Mocodes"),
    F.posexplode(F.split("Mocodes", " ")).alias("pos","MO_CODE")
 )\

Mocodes_split_df.show()

+---------+--------------------+------------------+---+-------+
|    DR_NO|           Race Desc|           Mocodes|pos|MO_CODE|
+---------+--------------------+------------------+---+-------+
|  1307355|Hispanic/Latin/Me...|[0913, 1814, 2000]|  0|   0913|
|  1307355|Hispanic/Latin/Me...|[0913, 1814, 2000]|  1|   1814|
|  1307355|Hispanic/Latin/Me...|[0913, 1814, 2000]|  2|   2000|
| 90631215|               White|[1100, 0400, 1402]|  0|   1100|
| 90631215|               White|[1100, 0400, 1402]|  1|   0400|
| 90631215|               White|[1100, 0400, 1402]|  2|   1402|
|100100501|Hispanic/Latin/Me...|            [0400]|  0|   0400|
|100100506|               Black|      [0344, 1402]|  0|   0344|
|100100506|               Black|      [0344, 1402]|  1|   1402|
|100100508|Hispanic/Latin/Me...|            [0344]|  0|   0344|
|100100509|               Black|            [0416]|  0|   0416|
|100100510|Hispanic/Latin/Me...|      [0400, 0416]|  0|   0400|
|100100510|Hispanic/Latin/Me...|      [0

In [117]:
display(Markdown("**Most frequent Motives for a Crime against Hispanic/Latin/Mexican**."))

Mocodes_tot_df = Mocodes_split_df\
    .where(col("Race Desc")=="Hispanic/Latin/Mexican")\
    .join(Mocodes_df,"MO_CODE")\
    .groupBy("MO_DESCRIPTION")\
    .agg(count("MO_DESCRIPTION").alias("Total Motive in Crimes"))\
    .orderBy(col("Total Motive in Crimes").desc())\
    .show(truncate = False)

**Most frequent Motives for a Crime against Hispanic/Latin/Mexican**.

+-----------------------------------------------+----------------------+
|MO_DESCRIPTION                                 |Total Motive in Crimes|
+-----------------------------------------------+----------------------+
|Removes vict property                          |221927                |
|Hit-Hit w/ weapon                              |137571                |
|Domestic violence                              |89936                 |
|Vandalized                                     |87439                 |
|Stranger                                       |72744                 |
|Force used                                     |61083                 |
|Victim knew Suspect                            |58035                 |
|Vehicle involved                               |47321                 |
|Evidence Booked (any crime)                    |41630                 |
|Pushed                                         |38609                 |
|Susp uses vehicle                              |36

# Machine Learning Model to Predict if crime is done to a hispanic

We decided to try with time segment, age, sex, Crm Cd and first motive to try to predict correctly a crime against hispanic. This implied more feature engineering and reversing previous transformations to be able to feed the model with the right type of data.

### Data Cleansing for model

In [118]:
Firstmotive_df = Mocodes_split_df.where(col("pos")==0)\
                                    .select("DR_NO","MO_CODE")#.show()

Crimes_df=SeverityCategorizationDF\
            .join(Firstmotive_df, "DR_NO")\
            #.select("MO_CODE").show()

In [119]:
Crimes_df.printSchema()

root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: date (nullable = true)
 |-- DATE OCC: date (nullable = true)
 |-- TIME OCC: timestamp (nullable = true)
 |-- AREA: integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 |-- LOC

In [120]:
MLdf = Crimes_df.select(col("TimeSegment"),\
                    col("Vict Age").cast("float"),\
                    col("Vict Sex"),\
                    col("Crm Cd").cast("float"),\
                    col("Vict Descent"),\
                    col("MO_CODE").cast("float"))

MLdf.show()

+---------------+--------+--------+------+------------+-------+
|    TimeSegment|Vict Age|Vict Sex|Crm Cd|Vict Descent|MO_CODE|
+---------------+--------+--------+------+------------+-------+
|1.Early Morning|    35.0|       F| 626.0|           B|  400.0|
|         6.Dawn|    41.0|       F| 740.0|           H|  329.0|
|      4.Evening|    47.0|       F| 230.0|           W|  411.0|
| 2.Late Morning|    28.0|       M| 647.0|           W|  400.0|
| 2.Late Morning|    45.0|       F| 624.0|           B|  416.0|
| 2.Late Morning|    36.0|       M| 649.0|           O|  800.0|
| 2.Late Morning|    46.0|       F| 330.0|           W|  344.0|
|      4.Evening|    60.0|       M| 420.0|           H|  344.0|
|      4.Evening|    24.0|       F| 624.0|           H|  400.0|
|      4.Evening|    51.0|       M| 745.0|           H|  329.0|
|      4.Evening|    20.0|       F| 946.0|           H|  329.0|
|    3.Afternoon|    16.0|       M| 930.0|           H|  421.0|
|        5.Night|    41.0|       F| 745.

In [121]:
#Check for nulls
MLdf.select([count(when(isnull(c), c)).alias(c) for c in MLdf.columns])\
            .show()

+-----------+--------+--------+------+------------+-------+
|TimeSegment|Vict Age|Vict Sex|Crm Cd|Vict Descent|MO_CODE|
+-----------+--------+--------+------+------------+-------+
|          0|       0|      57|     0|          83|      0|
+-----------+--------+--------+------+------------+-------+



In [122]:
MLdf_clean = MLdf.dropna(how="any")

### Feature Engineering

In [123]:
MLdf_hispanic = MLdf_clean.withColumn("Vict Descent", when(col("Vict Descent")=="H",1)\
                               .otherwise(0))
MLdf_hips_time = MLdf_hispanic.withColumn("TimeSegmentCode",when(col("TimeSegment")=="1.Early Morning",1)\
                                        .when(col("TimeSegment")=="2.Late Morning",2)\
                                        .when(col("TimeSegment")=="3.Afternoon",3)\
                                        .when(col("TimeSegment")=="4.Evening",4)\
                                        .when(col("TimeSegment")=="5.Night",5)\
                                        .otherwise(6))
MLdf_hips_time.show()

+---------------+--------+--------+------+------------+-------+---------------+
|    TimeSegment|Vict Age|Vict Sex|Crm Cd|Vict Descent|MO_CODE|TimeSegmentCode|
+---------------+--------+--------+------+------------+-------+---------------+
|1.Early Morning|    35.0|       F| 626.0|           0|  400.0|              1|
|         6.Dawn|    41.0|       F| 740.0|           1|  329.0|              6|
|      4.Evening|    47.0|       F| 230.0|           0|  411.0|              4|
| 2.Late Morning|    28.0|       M| 647.0|           0|  400.0|              2|
| 2.Late Morning|    45.0|       F| 624.0|           0|  416.0|              2|
| 2.Late Morning|    36.0|       M| 649.0|           0|  800.0|              2|
| 2.Late Morning|    46.0|       F| 330.0|           0|  344.0|              2|
|      4.Evening|    60.0|       M| 420.0|           1|  344.0|              4|
|      4.Evening|    24.0|       F| 624.0|           1|  400.0|              4|
|      4.Evening|    51.0|       M| 745.

In [124]:
from pyspark.ml.feature import StringIndexer

MLdf_model = StringIndexer(inputCol="Vict Sex",\
                            outputCol='Gender',\
                            handleInvalid='keep').fit(MLdf_hips_time).transform(MLdf_hips_time)


MLdf_model.show()

+---------------+--------+--------+------+------------+-------+---------------+------+
|    TimeSegment|Vict Age|Vict Sex|Crm Cd|Vict Descent|MO_CODE|TimeSegmentCode|Gender|
+---------------+--------+--------+------+------------+-------+---------------+------+
|1.Early Morning|    35.0|       F| 626.0|           0|  400.0|              1|   0.0|
|         6.Dawn|    41.0|       F| 740.0|           1|  329.0|              6|   0.0|
|      4.Evening|    47.0|       F| 230.0|           0|  411.0|              4|   0.0|
| 2.Late Morning|    28.0|       M| 647.0|           0|  400.0|              2|   1.0|
| 2.Late Morning|    45.0|       F| 624.0|           0|  416.0|              2|   0.0|
| 2.Late Morning|    36.0|       M| 649.0|           0|  800.0|              2|   1.0|
| 2.Late Morning|    46.0|       F| 330.0|           0|  344.0|              2|   0.0|
|      4.Evening|    60.0|       M| 420.0|           1|  344.0|              4|   1.0|
|      4.Evening|    24.0|       F| 624.0| 

In [125]:
#check for datatypes
MLdf_model=MLdf_model.drop("Vict Sex","TimeSegment")
MLdf_model.dtypes

[('Vict Age', 'float'),
 ('Crm Cd', 'float'),
 ('Vict Descent', 'int'),
 ('MO_CODE', 'float'),
 ('TimeSegmentCode', 'int'),
 ('Gender', 'double')]

In [126]:
#assemnling features into one single list
from pyspark.ml.feature import VectorAssembler

required_features = ['TimeSegmentCode','Vict Age','Gender','Crm Cd','MO_CODE']
assembler = VectorAssembler(inputCols=required_features,\
                            outputCol='features')
ML_featuresDF = assembler.transform(MLdf_model)

ML_featuresDF.show()

+--------+------+------------+-------+---------------+------+--------------------+
|Vict Age|Crm Cd|Vict Descent|MO_CODE|TimeSegmentCode|Gender|            features|
+--------+------+------------+-------+---------------+------+--------------------+
|    35.0| 626.0|           0|  400.0|              1|   0.0|[1.0,35.0,0.0,626...|
|    41.0| 740.0|           1|  329.0|              6|   0.0|[6.0,41.0,0.0,740...|
|    47.0| 230.0|           0|  411.0|              4|   0.0|[4.0,47.0,0.0,230...|
|    28.0| 647.0|           0|  400.0|              2|   1.0|[2.0,28.0,1.0,647...|
|    45.0| 624.0|           0|  416.0|              2|   0.0|[2.0,45.0,0.0,624...|
|    36.0| 649.0|           0|  800.0|              2|   1.0|[2.0,36.0,1.0,649...|
|    46.0| 330.0|           0|  344.0|              2|   0.0|[2.0,46.0,0.0,330...|
|    60.0| 420.0|           1|  344.0|              4|   1.0|[4.0,60.0,1.0,420...|
|    24.0| 624.0|           1|  400.0|              4|   0.0|[4.0,24.0,0.0,624...|
|   

### Model Training - Random Forest Classifier


In [127]:
#Train and test set
from pyspark.ml.classification import RandomForestClassifier

# 1. Split the current data set to get the training (80%) and test (20%) data sets.
(trainingDF, testDF) = ML_featuresDF.randomSplit([0.8,0.2])

# 2. Initialize the algorithm to understand our featured data set.
rfcAlgorithm = RandomForestClassifier(labelCol='Vict Descent',\
                                      featuresCol='features',\
                                      maxDepth=5)

#3. Train the algorithm to build the model and apply it on the test data set.
model = rfcAlgorithm.fit(trainingDF)
predictions = model.transform(testDF)
predictions.dtypes

[('Vict Age', 'float'),
 ('Crm Cd', 'float'),
 ('Vict Descent', 'int'),
 ('MO_CODE', 'float'),
 ('TimeSegmentCode', 'int'),
 ('Gender', 'double'),
 ('features', 'vector'),
 ('rawPrediction', 'vector'),
 ('probability', 'vector'),
 ('prediction', 'double')]

### Model Evaluation

In [131]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='Vict Descent',\
                                              predictionCol='prediction',\
                                              metricName='recallByLabel')
precision = evaluator.evaluate(predictions)
print('The recall of our model is ', precision)

The recall of our model is  0.9048349736602911


In [132]:
TotalCrimeHisp = SeverityCategorizationDF.where(col("Race Desc")=="Hispanic/Latin/Mexican").count()

TotalCrime= SeverityCategorizationDF.count()

TotalCrimeHisp/TotalCrime

0.40357380030562834

# Conclusion

We managed to obtain a model with 89% of recall, this is a great improvement taking into consideration that we have 40% of hispanic victims in our dataset. This is evidence enough to conclude that this variables are capable to predict a crime agianst a hispanic, and that there are opportunities to stop this behavior and prevent more racial crime relates in LA.